# Assignment 8 - Acoustic Localisation By EE22B025

### INSTRUCTIONS TO RUN CODE : 
- The modules `numpy` and `matplotlib` must be installed.
- When running the `rx2` and `rx3` datasets, change the value of `C` to 0.5.  
#### THANK YOU

In [ ]:
# Setting up imports 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Main system parameters: number of mics, number of samples in time
Nmics = 64
Nsamp = 200

# Source: x,y coordinates: x: 0+, y: [-Y, +Y] where Y determined by pitch and Nmics
src = (0, 0)

# Spacing between microphones
pitch = 0.1
# proxy for sampling rate
dist_per_samp = 0.1
# Speed of sound in the medium
C = 2
# Time dilation factor for sinc pulse: how narrow
SincP = 2.5

# CODE Locations of microphones
# X - coordinate of all these mics are 0

mics = []
if Nmics %2 == 0: #even number of mics
    y_coord = -pitch*(Nmics/2) 
    for j in range(Nmics):
        if j == 4:
            y_coord += pitch
        mics.append((0 , y_coord)) # as x-coordinate is 0
        y_coord += pitch
else:
    y_coord = -pitch*((Nmics-1)/2) 
    for j in range(Nmics):
        mics.append((0, y_coord)) # as x-coordinate is 0
        y_coord += pitch

# Location of point obstacle
obstacle = (3, -1)

In [ ]:
# Source sound wave - time axis and wave
# sinc wave with narrowness determined by parameter
time_limit = (Nsamp*dist_per_samp / C)
t = np.arange(0 ,time_limit , dist_per_samp) #time axis
def wsrc(t):
    return np.sinc(SincP*t)
ybase = wsrc(t)

plt.plot(t , ybase)
plt.show()

#### QUESTION: 
The plots below show two example sinc pulses. How will you generate pulses that look
like this? Which parameter should be changed? What effect do you think this will have on the final
image?

#### Answer: 
- You can generate pulses which look similar to the ones given in the plot by using the `wrsc(t)` function defined above, which is basically the `sinc` function.
- Make the value of `C` = 2.
- SincP is the parameter you need to change to get different types of pulse shape.
- To get the first figure, you need to change the SincP parameter to `1.0`. SincP is the dilation factor of the sinc function, which means that it decides how strechted or how compressed the sinc function would look like. 
- For figure 1, we needed a less compressed version of sinc, hence we reduced the SincP factor to 1.0
- For figure 2, we need a much more compressed version of sinc, hence we increase the SincP parameter to `5.0`
- You can also alter `Nsamp` and `dist_per_samp` values to alter the graph but the main parameter which determines the shape of the graph is `SincP`.

In [ ]:
# Cartesian distance between 2 points in xy plane
def cart_dist(x1, y1, x2, y2):
    return np.sqrt( (x1 - x2)**2 + (y1 - y2)**2 )
 

# Distance from src to a mic after reflecting through pt
def dist(src, pt, mic):
    d1 = cart_dist(src[0], src[1] , pt[0], pt[1]) # distance from src to pt
    d2 = cart_dist(pt[0], pt[1] , mic[0], mic[1]) # distance from pt to mic
    return d1 + d2

In [ ]:
plt.clf()

# Code for generating samples for each mic for a given obstacle
def time_delay(src, obstacle, mic):
    return ((dist(src,obstacle, mic))/ C ) #total distance divided by speed

ybase_delayed = [0 for x in range(Nmics)] # will contain all the delayed pulses

offset = 0 #to differentiate the plot of all mics

for j in range(Nmics):
    delay_time_obs = time_delay(src, obstacle, mics[j]) #delay time for obstacle
    ybase_delayed[j] = wsrc(t-delay_time_obs)/5 #scaling down to fit multiple curves
    plt.plot(t , ybase_delayed[j]+offset)
    offset = offset + 0.05

In [ ]:
# showing HEAT MAP 
two_D_array = np.vstack(ybase_delayed)
plt.imshow( two_D_array, aspect='auto', interpolation='nearest')

### DELAY AND SUM ALGORITHM

In [ ]:
final_image = np.zeros((Nmics, len(t))) #Final reconstructed image

for i in range(len(t)):
    for k in range(len(mics)):
        mic_sample = 0
        for m in mics:
            delay_value = time_delay(src, (t[i],mics[k][1]) , m) #getting delay value for each mic
            
            if delay_value < time_limit : #delay should be within sample range
                
                index_val = int(delay_value/dist_per_samp) #getting the corresponding index for delay_value
                temp = ybase_delayed[k][index_val]
                mic_sample += temp
                
        final_image[k][i] = mic_sample

plt.imshow( final_image, aspect='auto', interpolation='nearest')

#### QUESTIONS : 
1) Does it make sense to reconstruct up to `Nsamp`? What value is more reasonable as an upper limit for the x-axis here?

    `Answer`: No, the more reasonable upper limit for x axis here is `(Nsamp*dist_per_samp)/C`. This value is the upper limit of the time sample. This is more reasonable because we have contructed the source function using `time samples` as x-axis parameter, so it better to use that instead of just `Nsamp`.

2) The (x, y) coordinates corresponding to the maximum amplitude (yellow colour) is approximately (30, 22). Explain why this is the correct expected position for the given obstacle

    `Answer`: This is the correct expected position. The X coordinate in the HEATMAP corresponds to the no. of `time` samples. Here there are total of 100 time samples with upper limit (`(Nsamp*dist_per_samp)/C`) as 10, so the x coordinate of the obstacle (3) will now correspond to 30. For the y-coordinate, we contructed the grid such that y-axis is (0,63) for the 64 Mics. Earlier it was -3.2 to +3.2 (mic position) but after scaling and shifting, we can see that the y-coordinate of the obstacle (-1) should corresponding to approximately 22 [`-1*10(scale) +32(shift)`]
    
3) What is the maximum obstacle x- and y- coordinate that you can use and still have an image reconstructed?
    
    `Answer`: You can take the x- coordinate upto the the max limit of `(Nsamp*dist_per_samp)/C`. As for the y- coordinate, as long as you are in the range of Nmics, you will get a decent reconstructed image.
    
4) What happens if C is different - if C is decreased it looks like the image becomes sharper. Can you explain why intuitively?

    `Answer`: If C is reduced, the sharpness increases because `(Nsamp*dist_per_samp)/C` increases and hence the number of time samples increase giving a more sharper image.
    
5) What happens if Nmics is increased or decreased? Do the experiments with Nmics = [8, 32, 64] and Nsamp = [50, 100, 200] (all combinations). Attach the resulting images.

    `Answer` in PDF!

### RX2.TXT AND RX3.TXT

NOTE: CHANGE THE VALUE OF `C` TO `0.5` BEFORE RUNNING THE 2 CELLS GIVEN BELOW

In [ ]:
# RX2.TXT

#CHANGE THE VALUE OF C to 0.5 before runnning!!

data_rx2 = np.loadtxt("rx2.txt")

final_image_rx2 = np.zeros((Nmics, len(t))) #Final reconstructed image
for i in range(len(t)):
    for k in range(len(mics)):
        mic_sample = 0
        for m in mics:
            delay_value = time_delay(src, (t[i],mics[k][1]) , m) #getting delay value for each mic
            
            if delay_value < time_limit : #delay should be within sample range
                
                index_val = int(delay_value/dist_per_samp)
                temp = data_rx2[k][index_val]
                mic_sample += temp
                
        final_image_rx2[k][i] = mic_sample

plt.imshow( final_image_rx2, aspect='auto', interpolation='nearest')


In [ ]:
# RX3.TXT

data_rx3 = np.loadtxt("rx3.txt")

final_image_rx3 = np.zeros((Nmics, len(t))) #Final reconstructed image
for i in range(len(t)):
    for k in range(len(mics)):
        mic_sample = 0
        for m in mics:
            delay_value = time_delay(src, (t[i],mics[k][1]) , m) #getting delay value for each mic
            
            if delay_value < time_limit : #delay should be within sample range
                
                index_val = int(delay_value/dist_per_samp)
                temp = data_rx3[k][index_val]
                mic_sample += temp
                
        final_image_rx3[k][i] = mic_sample

plt.imshow( final_image_rx3, aspect='auto', interpolation='nearest')